# Polynomial Harmonization algorithm

## Import libraries

In [1]:
import os       # using operating system dependent functionality (folders)
import glob
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew

import sys
sys.path.insert(0, '../') # path to our library functions
# from cvasl import file_handler as fh # 
# from cvasl import mold #
# from cvasl import carve
from cvasl import seperated
# from cvasl.file_handler import Config

### get data

In [2]:
filepath_mri = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath_mri,'StrokeMRI_pvc2c.csv') 

filepath_top = '../open_work/internal_results/cleaned_pvc2s/' 
filename_top = os.path.join(filepath_top,'TOP_pvc2c.csv') 

In [3]:
TOP = pd.read_csv(filename_top)
StrokeMRI = pd.read_csv(filename_mri)

In [4]:
TOP = TOP.drop(TOP.columns[0],axis=1)
#TOP
StrokeMRI = StrokeMRI.drop(StrokeMRI.columns[0],axis=1)
#StrokeMRI

In [5]:
TOP

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_ivc_ratio,gmwm_ivc_ratio,wmh_vol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b,aca_b,mca_b,pca_b,totalgm_b
0,sub-0001_1_ses-1_run-1,43.49,M,0.71736,0.52803,0.31812,0.45881,0.45881,1.743,24.0,8.7620,2.2748,2.3205,2.6858,2.4316,22.1022,80.4744,68.3224,52.4614,60.6981
1,sub-0002_1_ses-1_run-1,38.30,F,0.72383,0.62394,0.25673,0.45112,0.45112,1.629,23.0,9.0749,1.7564,2.3989,1.5982,1.9738,23.5401,87.3972,78.0359,63.9932,71.6047
2,sub-0019_1_ses-1_run-1,32.30,M,0.71224,0.53295,0.33594,0.45046,0.45046,0.621,13.0,8.8791,1.9850,1.8702,2.1648,2.1723,27.5573,94.0855,86.3816,62.6012,74.0588
3,sub-0020_1_ses-1_run-1,21.97,F,0.67042,0.43458,0.25974,0.49124,0.49124,0.736,23.0,6.2080,1.8657,1.9092,1.6402,1.8734,27.1813,100.3259,90.7345,68.0812,79.1128
4,sub-0022_1_ses-1_run-1,37.52,F,0.63666,0.46938,0.22144,0.47960,0.47960,2.009,22.0,7.4561,1.6049,2.0383,1.7943,1.7936,25.4706,86.7965,77.5578,69.0419,73.3492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,sub-1163_1_ses-1_run-1,19.06,F,0.78201,0.49489,0.24608,0.51347,0.51347,1.431,18.0,8.2103,1.9598,2.1176,2.2812,2.1436,29.0297,117.4774,107.4787,84.1319,96.4466
523,sub-1165_1_ses-1_run-1,33.86,F,0.71537,0.48601,0.27123,0.48578,0.48578,1.425,18.0,6.6858,1.4851,1.9477,1.4834,1.8447,25.7284,87.0537,85.7033,66.4337,77.1030
524,sub-1167_1_ses-1_run-1,33.92,M,0.73473,0.53549,0.35985,0.45074,0.45074,3.935,20.0,4.5665,1.8432,2.0176,2.0263,1.9480,21.4760,68.3383,63.4410,43.3879,54.4204
525,sub-1168_1_ses-1_run-1,45.31,M,0.71094,0.60974,0.37480,0.41931,0.41931,4.170,26.0,9.5267,1.7798,1.9444,1.7670,1.8076,21.0647,61.4625,56.6210,43.5347,51.4417


In [6]:
StrokeMRI

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_ivc_ratio,gmwm_ivc_ratio,wmh_vol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b,aca_b,mca_b,pca_b,totalgm_b
0,sub-59082_1_ses-1_run-1,43.172603,F,0.64754,0.49441,0.31320,0.44500,0.44500,1.249,24.0,8.0434,1.7273,1.9776,1.7141,1.9140,26.8262,104.3155,90.5047,73.8758,84.4527
1,sub-59083_1_ses-1_run-1,66.367123,F,0.60517,0.48594,0.42304,0.39968,0.39968,14.597,25.0,3.7791,1.9669,1.7154,2.6394,1.8815,24.9597,75.7051,65.3032,65.6771,67.1158
2,sub-59085_1_ses-1_run-1,55.838356,F,0.61724,0.53779,0.33692,0.41371,0.41371,6.341,30.0,5.1248,1.8146,1.9845,2.0827,1.9501,27.7117,81.0966,72.9980,60.8317,64.3183
3,sub-59086_1_ses-1_run-1,48.238356,F,0.63402,0.48542,0.29969,0.44677,0.44677,3.070,23.0,5.0993,1.6616,1.9321,2.0319,1.7644,24.9029,75.6702,70.6205,55.1641,61.8921
4,sub-59087_1_ses-1_run-1,58.616438,F,0.57348,0.45642,0.35190,0.41502,0.41502,1.761,23.0,6.6565,2.0185,1.8745,2.0016,1.9140,18.5264,63.3017,53.8047,44.1930,49.1181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,sub-59440_1_ses-1_run-1,73.928767,M,0.57259,0.52662,0.51982,0.35366,0.35366,26.949,64.0,2.2578,2.0305,2.0193,1.9228,2.0773,27.9190,82.7343,72.6401,54.3780,59.3805
510,sub-59440_2_ses-2_run-1,74.769863,M,0.57528,0.50907,0.52840,0.35671,0.35671,22.915,32.0,2.3344,2.5824,2.6416,2.2723,2.6217,28.7051,82.7827,76.7499,65.4477,61.8818
511,sub-59441_2_ses-2_run-1,74.512329,M,0.58675,0.47585,0.42899,0.39337,0.39337,5.249,30.0,3.8390,1.8317,1.8702,1.6799,1.7434,27.1356,66.0938,63.8049,52.7282,54.3541
512,sub-59442_1_ses-1_run-1,67.526027,M,0.63929,0.55235,0.39661,0.40251,0.40251,8.612,9.0,4.9303,1.6693,1.9822,2.1802,1.8634,24.0485,71.8750,65.1286,56.4845,59.2195


In [7]:
# identify polynomial columns
poly_columns = ['gm_vol', 'wm_vol', 'csf_vol',
       'gm_ivc_ratio', 'gmwm_ivc_ratio', 'wmh_vol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b', 'aca_b', 'mca_b', 'pca_b', 'totalgm_b']

In [8]:


def polyfit_second_degree_to_df(
        dataframe,
        special_column_name,
        other_column_names,
):
    """
    This function creates a polynomial for two columns.
    It returns the coefficients
    
    :param dataframe: dataframe variable
    :type dataframe: pandas.dataFrame
    :param special_column_name: string of column you want to graph against
    :type  special_column_name: str
    :param other_column_names: list of column you want to get poly coefficientson
    :type other_column_names: list



    :returns: coeffiects
    :rtype: :class:`~numpy.ndarray`
    """
    list_as = []
    list_bs = []
    list_cs = []
    list_columns = []
    dataframe = dataframe.dropna()
    for interest_column_name in other_column_names:
        xscat = np.array(pd.to_numeric(dataframe[special_column_name]))
        yscat = np.array(pd.to_numeric(dataframe[interest_column_name]))
        coefficients = np.polyfit(xscat, yscat, 2 ) #2 = degree_poly
        list_columns.append(interest_column_name)
        list_as.append(coefficients[0])
        list_bs.append(coefficients[1])
        list_cs.append(coefficients[2])
    d = {'column':list_columns,'coefficient_a':list_as, 'coefficient_b':list_bs, 'coefficient_c':list_cs}
    coefficient_dataframe = pd.DataFrame(d)
   
    return coefficient_dataframe


In [9]:
TOP_polys = polyfit_second_degree_to_df(
        TOP,
        'age',
        poly_columns,
)

In [10]:
MRI_polys = polyfit_second_degree_to_df(
        StrokeMRI,
        'age',
        poly_columns,
)
MRI_polys

,column,coefficient_a,coefficient_b,coefficient_c
0,gm_vol,0.000006,-0.003375,0.807184
1,wm_vol,-0.000037,0.003136,0.455146
2,csf_vol,0.000053,-0.001645,0.280681
3,gm_ivc_ratio,-0.000001,-0.001814,0.528254
4,gmwm_ivc_ratio,-0.000001,-0.001814,0.528254
5,wmh_vol,0.016750,-1.407991,30.300888
6,wmh_count,0.014756,-1.261558,47.341133
7,deepwm_b_cov,-0.001239,0.027087,8.134194
8,aca_b_cov,0.000321,-0.026994,2.374683
9,mca_b_cov,0.000150,-0.017337,2.482435


In [11]:
col_list = []
transformer_list = []
poly_columns = ['coefficient_a','coefficient_b','coefficient_c']
poly_columns_top = {
    original: 'top_{}'.format(original) for original in poly_columns
}
poly_columns_top_renamed = list(poly_columns_top.values())
poly_columns_mri = {
    original: 'mri_{}'.format(original) for original in poly_columns
}
poly_columns_mri_renamed = list(poly_columns_mri.values())
top_polys_selection = TOP_polys[poly_columns]
mri_polys_selection = MRI_polys[poly_columns]
together = top_polys_selection.copy()
together.rename(columns=poly_columns_top, inplace=True)
together = together.assign(**{
    col_name: MRI_polys[col_name] for col_name in poly_columns
})
together.rename(columns=poly_columns_mri, inplace=True)

def tf_matrix(row):
    return seperated.generate_transformation_matrix(row[poly_columns_top_renamed], row[poly_columns_mri_renamed])

transformers = together.apply(tf_matrix, axis=1)
transformers

0     [[-0.6388086395636249, 0.0, 0.0], [0.0, 1.4101...
1     [[0.44798103659571353, 0.0, 0.0], [0.0, 0.4853...
2     [[7.594923578545894, 0.0, 0.0], [0.0, -0.68852...
3     [[-0.05610616722828195, 0.0, 0.0], [0.0, 0.531...
4     [[-0.05610616722828195, 0.0, 0.0], [0.0, 0.531...
5     [[7.107245879083433, 0.0, 0.0], [0.0, 10.52405...
6     [[4.6855543000731545, 0.0, 0.0], [0.0, 8.13191...
7     [[0.6465173217300115, 0.0, 0.0], [0.0, 0.28692...
8     [[3.1733552214805147, 0.0, 0.0], [0.0, 3.12828...
9     [[0.8223380110079599, 0.0, 0.0], [0.0, 0.97204...
10    [[2.0308323825829357, 0.0, 0.0], [0.0, 3.44138...
11    [[1.625827149550833, 0.0, 0.0], [0.0, 1.601782...
12    [[0.0681849399401173, 0.0, 0.0], [0.0, -0.0215...
13    [[-0.48466881677487733, 0.0, 0.0], [0.0, -0.20...
14    [[-0.27442597172886607, 0.0, 0.0], [0.0, -0.06...
15    [[-0.6099978973901993, 0.0, 0.0], [0.0, -0.320...
16    [[-0.39300157445887896, 0.0, 0.0], [0.0, -0.09...
dtype: object

# Home made Harmonizartion aglorithm #1

1. take polynomials of top and mri
2. assume we want to harmonize all values as if they had been shot on the strokeMRI machine
3. take the difference betwen top point and top polynomial for each point
4. make harmonized TOp dataset by adding the differences to the stroke MRI polynomial (at the same age)
5. test the harmonized for 'ML harmony'